### Лабораторная работа №3

##### <span style="color:blue">Задание.</span> Провести классификацию найденного датасета, методами линеной и логистической регрессий . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Сначала мы импортируем необходимые библиотеки и нормализуем данные:

In [15]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

data = pd.read_csv("adult.data")
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country', 'income']
data.columns = column_names

data = data.replace(' ?', pd.NA)

data = data.dropna()
data = pd.get_dummies(data, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
data['income'] = data['income'].map({' <=50K': 0, ' >50K': 1})

Разделим данные на независимые переменные (X) и зависимую переменную (y) и проведем предварительную обработку данных (Преобразование категориальных признаков в числовые и разделение на обучающую и тестовую выборки):

In [16]:
X = data.drop('income', axis=1)
y = data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Обучение модели линейной регрессии

In [17]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

y_pred_linear = linear_model.predict(X_test)

Задаем сетки параметров и ищем лучшие для логистической регрессии:

In [18]:
logistic_model = LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

grid_search = GridSearchCV(logistic_model, param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Лучшие параметры: ", grid_search.best_params_)

Best parameters:  {'C': 0.001, 'penalty': 'l2'}


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1168, in fit
    solver = _

Обучение модели логистической регрессии

In [19]:
best_logistic_model = grid_search.best_estimator_

best_logistic_model.fit(X_train, y_train)
y_pred_logistic = best_logistic_model.predict(X_test)

Оценка точности

In [20]:
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
accuracy_linear = accuracy_score(y_test, y_pred_linear.round())

print("Точность модели линейной регрессии:", accuracy_linear)
print("Точность модели логистической регрессии:", accuracy_logistic)

Точность модели линейной регрессии: 0.8327531907840212
Точность модели логистической регрессии: 0.7938007624730649


Теперь у нас есть оценка точности для обеих моделей. В данном случае мы использовали параметры по умолчанию для линейной модели. При обучении модели логистической регрессии добавлен параметр max_iter=1000 для избежания предупреждений о сходимости модели.